In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import pie,axis,show
import seaborn as sns
import random as rd
import datetime as dt
from termcolor import colored

In [4]:
#my source
devices=pd.read_csv('../data/devices.csv')
users=pd.read_csv('../data/users.csv')

#github source 
#devices=pd.read_csv('../data/devices.csv')
#users=pd.read_csv('../data/users.csv')

In [5]:
print('the shape of devices is: ',devices.shape)
print('the shape of users is: ',users.shape)

the shape of devices is:  (15544, 2)
the shape of users is:  (15544, 12)


In [6]:
devices = devices.set_index('user_id')
df=pd.merge(users,devices,on='user_id')
userdf=df.set_index('user_id')
userdf.head(2)

,birth_year,country,city,created_date,user_settings_crypto_unlocked,plan,attributes_notifications_marketing_push,attributes_notifications_marketing_email,num_contacts,num_referrals,num_successful_referrals,brand
user_id,,,,,,,,,,,,
user_0,1992,ES,Madrid,2018-09-10 18:46:42.843855,0,STANDARD,1.0,1.0,3,0,0,Apple
user_1,1985,LT,Klaipeda,2018-01-11 18:53:10.531146,1,STANDARD,NaN,NaN,75,0,0,Apple


In [7]:
print(userdf.describe())
print(userdf.dtypes)
print(userdf.isnull().sum())

         birth_year  user_settings_crypto_unlocked  \
count  15544.000000                   15544.000000   
mean    1983.666624                       0.182385   
std       11.369184                       0.386174   
min     1929.000000                       0.000000   
25%     1977.000000                       0.000000   
50%     1986.000000                       0.000000   
75%     1992.000000                       0.000000   
max     2001.000000                       1.000000   

       attributes_notifications_marketing_push  \
count                             10284.000000   
mean                                  0.948269   
std                                   0.221494   
min                                   0.000000   
25%                                   1.000000   
50%                                   1.000000   
75%                                   1.000000   
max                                   1.000000   

       attributes_notifications_marketing_email  num_contacts 

##Encoding of users and devices

In [8]:
# Replace 'SILVER' and 'GOLD' with 'PAID' in order to have two categories of plan: 'STANDARD' and 'PAID'
userdf['plan'] = userdf['plan'].str.replace('SILVER', 'PAID')
userdf['plan'] = userdf['plan'].str.replace('GOLD', 'PAID')
print("Number of customers by plan ","\n" ,userdf['plan'].value_counts())
print(colored("Regarding the 15544 users, the 14394 or 92.6% of them are STANDARD and the rest 1150 or 7.3% are PAID","red" ))

Number of customers by plan  
 STANDARD    14394
PAID         1150
Name: plan, dtype: int64
Regarding the 15544 users, the 14394 or 92.6% of them are STANDARD and the rest 1150 or 7.3% are PAID


### Discovering Email and Push Acceptance

In [9]:
print(pd.crosstab(index=userdf['attributes_notifications_marketing_email'],columns=userdf['plan']))
print(colored("Regarding the concept about the acceptance of email:","red"))
print(colored("About the 830 paid users: 73 (6.3%) have not accepted, while 757 (65.8%) have accepted", "blue"))
print(colored("About the 9454 standard users: 1002 (6.9%) have not accepted, while 8452 (58.7%) have accepted","green" ))
print("\n")
print(pd.crosstab(index=userdf['attributes_notifications_marketing_email'].isnull(),columns=userdf['plan']))
print(colored("Regarding the paid users, 320 have Nan","blue"))
print(colored("Regarding the standard users, 4940 have Nan","green"))

plan                                      PAID  STANDARD
attributes_notifications_marketing_email                
0.0                                         73      1002
1.0                                        757      8452
Regarding the concept about the acceptance of email:
About the 830 paid users: 73 (6.3%) have not accepted, while 757 (65.8%) have accepted
About the 9454 standard users: 1002 (6.9%) have not accepted, while 8452 (58.7%) have accepted


plan                                      PAID  STANDARD
attributes_notifications_marketing_email                
False                                      830      9454
True                                       320      4940
Regarding the paid users, 320 have Nan
Regarding the standard users, 4940 have Nan


In [10]:
print(pd.crosstab(index=userdf['attributes_notifications_marketing_push'],columns=userdf['plan']))
print(colored("Regarding the concept about the acceptance of push:","red"))
print(colored("About the 830 paid users:  46 (5.5%) have not accepted, while 784 (94.5%) have accepted", "blue"))
print(colored("About the 9454 standard users: 486 (5%) have not accepted, while 8968 (95%) have accepted","green" ))
print("\n")
print(pd.crosstab(index=userdf['attributes_notifications_marketing_push'].isnull(),columns=userdf['plan']))
print(colored("Regarding the paid users, 320 have Nan","blue"))
print(colored("Regarding the standard users, 4940 have Nan","green"))

plan                                     PAID  STANDARD
attributes_notifications_marketing_push                
0.0                                        46       486
1.0                                       784      8968
Regarding the concept about the acceptance of push:
About the 830 paid users:  46 (5.5%) have not accepted, while 784 (94.5%) have accepted
About the 9454 standard users: 486 (5%) have not accepted, while 8968 (95%) have accepted


plan                                     PAID  STANDARD
attributes_notifications_marketing_push                
False                                     830      9454
True                                      320      4940
Regarding the paid users, 320 have Nan
Regarding the standard users, 4940 have Nan


###Filling Nan values in Email and Push Acceptance

In [11]:
##ara tha gemisw posostiaia tis nan times kai tha thewrisw oti apo tous 320 paid xristes me nan oi 292(91.25%) tha dextoun (=1) enw oi 28(8.75%) oxi (=0)
##kai oson afora tous standard apo tous 4940 me nan times tha thewrisw tous 4416(90%) oti tha dextoun (=1) enw oi 524(10%) oxi (=0) 
def mail(vec):
  prob=rd.random()
  plan, p = vec[0], vec[1]
  if plan=='STANDARD' and np.isnan(p):
    if prob<0.9:
      return 1
    else:
      return 0
  if plan=='PAID' and np.isnan(p):
    if prob<0.9125:
      return 1
    else:
      return 0
  return p   

In [12]:
userdf['attributes_notifications_marketing_email']=userdf[['plan','attributes_notifications_marketing_email']].apply(mail,axis=1)
print("Number of users that have accepted the email notifications (value 1) and the rest that have not accepted (value 0)","\n",userdf['attributes_notifications_marketing_email'].value_counts())

Number of users that have accepted the email notifications (value 1) and the rest that have not accepted (value 0) 
 1.0    13985
0.0     1559
Name: attributes_notifications_marketing_email, dtype: int64


In [13]:
##ara tha gemisw posostiaia tis nan times kai tha thewrisw oti apo tous 320 paid xristes me nan oi 305(95%) tha dextoun (=1) enw oi 15(5%) oxi (=0)
##kai oson afora tous standard apo tous 4940 me nan times tha thewrisw tous 4693(95%) oti tha dextoun (=1) enw oi 247(5%) oxi (=0)
def push(vec):
  prob=rd.random()
  plan, p = vec[0], vec[1]
  if plan=='STANDARD' and np.isnan(p):
    if prob<0.95:
      return 1
    else:
      return 0
  if plan=='PAID' and np.isnan(p):
    if prob<0.95:
      return 1
    else:
      return 0    
  return p   

In [14]:
userdf['attributes_notifications_marketing_push']=userdf[['plan','attributes_notifications_marketing_push']].apply(push,axis=1)
print("Number of users that have accepted the push notifications (value 1) and the rest that have not accepted (value 0)","\n",userdf['attributes_notifications_marketing_push'].value_counts())

Number of users that have accepted the push notifications (value 1) and the rest that have not accepted (value 0) 
 1.0    14737
0.0      807
Name: attributes_notifications_marketing_push, dtype: int64


In [15]:
userdf.dtypes

birth_year                                    int64
country                                      object
city                                         object
created_date                                 object
user_settings_crypto_unlocked                 int64
plan                                         object
attributes_notifications_marketing_push     float64
attributes_notifications_marketing_email    float64
num_contacts                                  int64
num_referrals                                 int64
num_successful_referrals                      int64
brand                                        object
dtype: object

###Calculating the age of the users

In [16]:
userdf['birth_year'] =2020 - (userdf[['birth_year']]) #transforming birth year
userdf.rename(columns={'birth_year':"age" }, inplace=True)

###Extracting the info about the month and year created and then make dummies for year

In [17]:
userdf['created_date'] = userdf['created_date'].apply(pd.to_datetime)
userdf['created_year']=pd.DatetimeIndex(userdf['created_date']).year
userdf['created_month']=pd.DatetimeIndex(userdf['created_date']).month
print("Number of customers by created year ","\n" ,userdf['created_year'].value_counts())
print("Number of customers by created month ","\n" ,userdf['created_month'].value_counts())

Number of customers by created year  
 2018    15494
2019       50
Name: created_year, dtype: int64
Number of customers by created month  
 12    1922
11    1555
10    1454
8     1421
3     1418
9     1407
5     1266
6     1262
7     1237
4      957
1      937
2      708
Name: created_month, dtype: int64


In [18]:
#Create the two dummies about the years 2018 and 2019
userdf=pd.concat([pd.get_dummies(userdf['created_year']), userdf], axis=1)

In [19]:
userdf.rename(columns={2018:"year_2018",2019:"year_2019" }, inplace=True)

###Handling the columns of referrals and the successful referrals

In [20]:
print("The number of total referrals", "\n",userdf['num_referrals'].value_counts())
print("The number of total successfull referrals", "\n",userdf['num_successful_referrals'].value_counts())
print(colored("All the observations about these two attributes have a zero value. Let's think about it... :(", "red"))

The number of total referrals 
 0    15544
Name: num_referrals, dtype: int64
The number of total successfull referrals 
 0    15544
Name: num_successful_referrals, dtype: int64
All the observations about these two attributes have a zero value. Let's think about it... :(


###Encoding the plan with 0 the STANDARD and 1 the PAID

In [21]:
#function to encode plan to 'paid' and 'standard'
def change(x):              
  if x == 'STANDARD':
    return 0
  elif x == 'PAID':
    return 1

In [22]:
userdf['plan']=userdf['plan'].apply(change)
print("The number of paid and standard users", "\n",userdf['plan'].value_counts())

The number of paid and standard users 
 0    14394
1     1150
Name: plan, dtype: int64


###Handling the brand of mobile phone

In [23]:
pd.crosstab(index=userdf['brand'],columns=userdf['plan'])

plan,0,1
brand,,
Android,7346,454
Apple,7025,687
Unknown,23,9


In [24]:
#Only 32 obs of unknown, so we have to replace it somehow
#Replace 'Unknown' with 'Android' in order to have two big categories of brand: 'Android' and 'Apple'
userdf['brand'] = userdf['brand'].str.replace('Unknown', 'Android')
userdf['brand'].value_counts()

Android    7832
Apple      7712
Name: brand, dtype: int64

In [25]:
#Now make a dummie about devices, Android=0 and Apple=1
brand_dict = {'Android': 0, 'Apple': 1}
userdf['brand'] = userdf['brand'].map(brand_dict)
userdf['brand'].value_counts()

0    7832
1    7712
Name: brand, dtype: int64

###Handling the countries and cities

In [26]:
print("Unique values of countries:","\n",userdf['country'].unique())
print("Unique values of cities:","\n",userdf['country'].unique())

Unique values of countries: 
 ['ES' 'LT' 'IE' 'GB' 'MT' 'FR' 'RO' 'PL' 'AT' 'IT' 'SI' 'CZ' 'BE' 'BG'
 'GI' 'CH' 'GG' 'CY' 'LV' 'DE' 'SK' 'GR' 'DK' 'PT' 'LU' 'HR' 'NL' 'NO'
 'SE' 'AU' 'IM' 'FI' 'EE' 'JE' 'HU' 'RE' 'IS' 'GP' 'LI' 'MQ']
Unique values of cities: 
 ['ES' 'LT' 'IE' 'GB' 'MT' 'FR' 'RO' 'PL' 'AT' 'IT' 'SI' 'CZ' 'BE' 'BG'
 'GI' 'CH' 'GG' 'CY' 'LV' 'DE' 'SK' 'GR' 'DK' 'PT' 'LU' 'HR' 'NL' 'NO'
 'SE' 'AU' 'IM' 'FI' 'EE' 'JE' 'HU' 'RE' 'IS' 'GP' 'LI' 'MQ']


In [27]:
#function to cluster the countries
def change_country(x):     
  if x=='AT' or x=='BE' or x=='DE' or x=='LU' or x=='NL' or x=='LI' or x=='CH':
    return 'Central_Rich_Europe'
  elif x=='IE' or x=='GB' or x=='GG' or x=='IM' or x=='JE' or x=='GI' or x=='DK' or x=='NO' or x=='SE' or x=='FI' or x=='IS':
    return 'North_Europe_Scand'
  elif x=='FR' or x=='RE' or x=='GP' or x=='MQ' or x=='MT' or x=='IT' or x=='CY' or x=='ES' or x=='PT':
    return 'Mediteranian_Europe'
  elif x=='EE' or x=='SK' or x=='HU' or x=='LV' or x=='SI' or x=='BG' or x=='GR' or x=='HR' or x=='LT' or x=='RO' or x=='PL' or x=='CZ':
    return 'East_Europe_Balkans'
  elif x=='AU':
    return 'Australia'

In [28]:
userdf['country']=userdf['country'].apply(change_country)
print("The number of users per region:","\n",userdf['country'].value_counts())

The number of users per region: 
 North_Europe_Scand     6408
East_Europe_Balkans    4268
Mediteranian_Europe    3818
Central_Rich_Europe    1046
Australia                 4
Name: country, dtype: int64


In [29]:
#Create dummies of each region
userdf=pd.concat([pd.get_dummies(userdf['country']), userdf], axis=1)

In [30]:
userdf.head(3)

,Australia,Central_Rich_Europe,East_Europe_Balkans,Mediteranian_Europe,North_Europe_Scand,year_2018,year_2019,age,country,city,...,user_settings_crypto_unlocked,plan,attributes_notifications_marketing_push,attributes_notifications_marketing_email,num_contacts,num_referrals,num_successful_referrals,brand,created_year,created_month
user_id,,,,,,,,,,,,,,,,,,,,,
user_0,0,0,0,1,0,1,0,28,Mediteranian_Europe,Madrid,...,0,0,1.0,1.0,3,0,0,1,2018,9
user_1,0,0,1,0,0,1,0,35,East_Europe_Balkans,Klaipeda,...,1,0,1.0,1.0,75,0,0,1,2018,1
user_2,0,0,0,0,1,1,0,24,North_Europe_Scand,Carrickmacross,...,0,0,1.0,0.0,71,0,0,1,2018,11


In [33]:
#if you run it locally
userdf.to_csv("../processed_data/users_devices.csv")

In [32]:
#if you run it in colab
# userdf.to_csv("users_devices.csv")